In [6]:
from datetime import datetime
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from tqdm import tqdm
import time
# Custom API built off of Prop-Odds and Underdog API
from OddsAPI import API
api = API()

api_keys = iter([
    'FdbjrrnSeqjP1AyfWCFLm0BV9ogkykWbjxh6r9Vxe6g',
    'VnB2yYMMlbYCtRGOp5O474Lvs9zQaQLKMPY2IaQZqeY',
    'YiJ0c0wp6gi9VxrSFvT3DcDCmInwMaqyqRDfLRkMqo',
    'awY7TnqX120wfiBjqCWUpleoycTdjBBWrhBghvdY3o4',
    'qT1Y5qCKTBfyjgcgwmpeQUzhceiezzmVujciuSJOTg',
    'bcAaq0xd8KlMKfQMVreZaz6SODB2CqioZmNqNIw',
    'DIbjrARXpgFpKQF8H6uOYWSOBDwGhqj4acEeOIyYa10',
    'mcVdtfKuncfWan5DHKZ5RLglM2cwsAzPi9jX7yu8',
    'rJmjk1h96mKbZumOJDeAHQKU9FNhzrMDX8NpPOsxd1g',
    'w4jbZx8umTb0cXSe363EQT92M0zwdvNCAp3UVMNaFSc',
    'TCxSdaWzcMaTibPSqrD0XjHmWRgyE5aHWyfaTaKdPs',
    'HAzwYt7XgbqsCfiL7IJqcteMc7rdd8gqxGsKqdjGA',
    'uiAPl9kIyidxUbsiLMYdWtoo1EShhvNBxxEkrQ9j9Wg',
    'P742pehVEUpSx63Fyg73lsCLwHl4eDjCK1IemgAYzE',
    'JXs9qfuzdcvl3fqlvvQvti1ac9ft35U2L2gbArOuds'
])

## prop-odds API Key
api.setAPI('HAzwYt7XgbqsCfiL7IJqcteMc7rdd8gqxGsKqdjGA')

In [7]:
## display 100 rows|
pd.set_option('display.max_rows', 100)

In [8]:
## pivot: converts a dataframe that contains two seperate rows for each player (one for over, one for under) to have one row for each player
def pivot(df):
    pivot_df = pd.pivot_table(df, values='odds', index=['handicap', 'participant_name'], columns='name').reset_index()
    pivot_df.columns = ['line', 'participant_name', 'over_odds', 'under_odds']
    return pivot_df

## calculate odds
## adapted from Ammar Sulmanjee
def calculate_odds(x, y=None):
    if not y:
        if x>= 0: return 1 + x/100
        return 1 + 100/abs(x)
    
    ## calculate total odds
    imp_prob1 = (1 / calculate_odds(x)) * 100
    imp_prob2 = (1 / calculate_odds(y)) * 100

    ## remove VIG
    total_implied_prob = round(imp_prob1 + imp_prob2, 4)
    fair_prob1 = round(imp_prob1 / total_implied_prob * 100, 2)
    fair_prob2 = round(imp_prob2 / total_implied_prob * 100, 2)

    return [fair_prob1, fair_prob2]

## saves best bets
def download_bets(best_bets):
    ## format (take out uneccesary parts of market string)
    best_bets['market'] = best_bets['market'].str.replace(r'_|player|over_under', '', regex=True).str.strip()
    best_bets = best_bets.drop(columns=['game'])
    best_bets.to_csv(f"Bets_{datetime.now().strftime('%Y-%m-%d')}.csv", index=None)

## check palyer's raw data (debugging function)
def check_player(raw_bet_data, name, market, line=None):
    if line:
        display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market) & (raw_bet_data['handicap'] == line)])
        return
    display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market)])

def check_participants(df):
    # Count occurrences of each participant_name
    counts = df['participant_name'].value_counts()

    # Filter participants whose name appears twice
    participants_twice = counts[counts == 2].index
    df_filtered = df[df['participant_name'].isin(participants_twice)]

    return df_filtered


In [9]:
### Pull Underdog Lines
ud = api.get_fantasy_lines()

## get markets and games for the lines
markets = set(ud['market'])

games = api.get_nba_games()
games = [i['game_id'] for i in games['games']]

In [10]:
## save formatted best bets, and raw data to for check_player() function
bets = pd.DataFrame()
raw_bet_data = pd.DataFrame()


## run though each game and each market, tqdm is for progress bar
for game in tqdm(games, desc="Processing games"):
    for market in markets:
            
        # get sportsbook odds
        try:
            bookies_data = api.get_most_recent_odds(game, market)
        except Exception as e:
            if str(e) == "ERROR 422": continue
            else: 
                print(e)
                continue
             
        # format each books data     
        books = []
        for i in range(len(bookies_data['sportsbooks'])):
            df = pd.DataFrame.from_dict(bookies_data['sportsbooks'][i]['market']['outcomes'])
            df = df[df['timestamp'].str.contains(datetime.now().strftime('%Y-%m-%d'))]

            df['book'] = bookies_data['sportsbooks'][i]['bookie_key']
            df['market'] = market
            df['name'] = df['name'].apply(lambda x: "over" if "over" in x.lower() else "under")

            books.append(df)

        # add to raw_data for checking
        if len(books) > 0:
            raw_data = pd.concat(books)
            raw_bet_data = pd.concat([raw_bet_data, raw_data])

        # for each book, only get lines that match with under dog.
        # then conat all of these lines + odds
        new_datasets = []
        for dataset in books:
            dataset = pd.merge(dataset[['handicap', 'odds', 'participant_name', 'name', 'market']], ud, how='inner', on = ['participant_name', 'handicap' , 'market'])
            dataset = dataset[['handicap', 'odds', 'participant_name', 'name']]
            dataset = check_participants(dataset)
            
            if dataset.shape[0] > 0 and dataset.shape[1] > 0:
                # converts 2 rows (one over, one under) into 1 column for each player
                dataset = pivot(dataset)
                new_datasets.append(dataset)
        if new_datasets == []: continue

        ## concat all of these bets together
        final = pd.concat(new_datasets, axis=0)
        final = final.dropna()

        ## calculate median odds
        final[['over_odds', 'under_odds']] = final.apply(lambda row: pd.Series(calculate_odds(row['over_odds'], row['under_odds'])), axis=1)
        final = final.groupby(['participant_name', 'line']).agg({'over_odds':'median', 'under_odds':'median'}).reset_index()

        ## find ev, add market and game for each player
        final['ev'] = abs(final['over_odds'] - 50)
        final['market'] = market
        final['game'] = game


        if final.shape[0] > 0: 
            bets = pd.concat([bets, final[['participant_name', 'ev', 'market', 'line', 'over_odds', 'under_odds', 'game']]])
            # if bets[bets['ev'] > 4.5].shape[0] > 0: display(bets[bets['ev'] > 4.5].sort_values(by=['ev'], ascending=False))

Processing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing games: 100%|██████████| 10/10 [00:40<00:00,  4.08s/it]


In [8]:
bets = bets.sort_values(by=['ev'], ascending=False)
bets = bets.dropna()
bets = bets.drop_duplicates(keep='first', ignore_index=True)

best_bets = bets[bets['ev'] > 4.5]

print(f"Total Bets found!: {bets.shape[0]}")
print(f"Total Positive EV Bets found!: {best_bets.shape[0]}")

Total Bets found!: 484
Total Positive EV Bets found!: 0


In [9]:
bets

,participant_name,ev,market,line,over_odds,under_odds,game
0,Paolo Banchero,4.080,player_rebounds_over_under,7.5,45.920,54.080,b845c095fb257ca1620b867c4d9de530
1,Zion Williamson,3.990,player_points_over_under,22.5,53.990,46.010,02e0bf8d3b5c84685db2a220fc0c97b5
2,Khris Middleton,3.950,player_rebounds_over_under,4.5,46.050,53.950,753b65609d24e43a89ef67b661c38b51
3,Wendell Carter Jr.,3.790,player_points_over_under,11.5,46.210,53.790,b845c095fb257ca1620b867c4d9de530
4,Tim Hardaway Jr.,3.745,player_steals_over_under,0.5,53.745,46.255,f7547a3dddc3bd8176be92b3a5fffd98
...,...,...,...,...,...,...,...
479,Jimmy Butler,0.000,player_assists_over_under,4.5,50.000,50.000,c96521dde3008893e0f55e16327596c1
480,Damian Lillard,0.000,player_points_rebounds_over_under,30.5,50.000,50.000,753b65609d24e43a89ef67b661c38b51
481,Donovan Mitchell,0.000,player_assists_over_under,6.5,50.000,50.000,35ad645e74f40025f5e9f33669f4f476
482,Keegan Murray,0.000,player_rebounds_over_under,5.5,50.000,50.000,c96521dde3008893e0f55e16327596c1


In [70]:
check_player(raw_bet_data, 'Nic Claxton', 'player_points_over_under', 14.5)

,timestamp,handicap,odds,participant,participant_name,name,description,book,market
7,2024-01-31T15:00:15,14.5,102,16552,Nic Claxton,over,Over Nic Claxton (Points),pinnacle,player_points_over_under
15,2024-01-31T15:00:15,14.5,-135,16552,Nic Claxton,under,Under Nic Claxton (Points),pinnacle,player_points_over_under
59,2024-01-31T14:44:12,14.5,114,16552,Nic Claxton,over,Over - Nic Claxton Points scored by the player...,betrivers,player_points_over_under
121,2024-01-31T14:44:12,14.5,-155,16552,Nic Claxton,under,Under - Nic Claxton Points scored by the playe...,betrivers,player_points_over_under
